In [ ]:
# 1. Autoriser ce notebook à accéder à vos fichiers Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

#lire le fichier
file_path = '/content/drive/MyDrive/DataAnalyst-mini-projects/Data/Customer Call List.xlsx'

df = pd.read_excel(file_path)
df



,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact,Not_Useful_Column
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",Yes,No,True
1,1002,Abed,Nadir,123/643/9775,93 West Main Street,No,Yes,False
2,1003,Walter,/White,7066950392,298 Drugs Driveway,N,NaN,True
3,1004,Dwight,Schrute,123-543-2345,"980 Paper Avenue, Pennsylvania, 18503",Yes,Y,True
4,1005,Jon,Snow,876|678|3469,123 Dragons Road,Y,No,True
5,1006,Ron,Swanson,304-762-2467,768 City Parkway,Yes,Yes,True
6,1007,Jeff,Winger,NaN,1209 South Street,No,No,False
7,1008,Sherlock,Holmes,876|678|3469,98 Clue Drive,N,No,False
8,1009,Gandalf,NaN,N/a,123 Middle Earth,Yes,NaN,False
9,1010,Peter,Parker,123-545-5421,"25th Main Street, New York",Yes,No,True


# *I.* Data Cleaning

In [ ]:
# 1 - Supprimer les doublons
df = df.drop_duplicates()


# 2 - Supprimer les colonnes non utiles (sans erreur si elle n'existe pas)
df = df.drop(columns=["Not_Useful_Column"], errors="ignore")


# 3 - Nettoyer Last_Name (caractères excédentaires)
df["Last_Name"] = df["Last_Name"].astype(str).str.strip("123._/")


# 3 - Nettoyer et formater Phone_Number
df["Phone_Number"] = (
    df["Phone_Number"]
    .astype(str)
    .str.replace('[^0-9]', '', regex=True)   # garder uniquement les chiffres
    .str.pad(10, fillchar='0')               # sécuriser la longueur
    .apply(lambda x: f"{x[0:3]}-{x[3:6]}-{x[6:10]}")
    .replace(['nan--', 'Na--'], '')          # nettoyer numéros invalides
)


# 4 - Séparer la colonne Address en 3 nouvelles colonnes
df[["Street_Address", "State", "Zip_Code"]] = (
    df["Address"]
    .astype(str)
    .str.split(pat=",", n=2, expand=True)
)

df["Do_Not_Contact"] = df["Do_Not_Contact"].str.replace('Yes','Y')
df["Do_Not_Contact"] = df["Do_Not_Contact"].str.replace('No','N')

df["Paying Customer"] = df["Do_Not_Contact"].str.replace('Yes','Y')
df["Paying Customer"] = df["Do_Not_Contact"].str.replace('No','N')

df = df.replace('N/a','NaN')




df

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact,Street_Address,State,Zip_Code
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",N,N,123 Shire Lane,Shire,None
1,1002,Abed,Nadir,123-643-9775,93 West Main Street,Y,Y,93 West Main Street,None,None
2,1003,Walter,White,706-695-0392,298 Drugs Driveway,NaN,NaN,298 Drugs Driveway,None,None
3,1004,Dwight,Schrute,123-543-2345,"980 Paper Avenue, Pennsylvania, 18503",Y,Y,980 Paper Avenue,Pennsylvania,18503
4,1005,Jon,Snow,876-678-3469,123 Dragons Road,N,N,123 Dragons Road,None,None
5,1006,Ron,Swanson,304-762-2467,768 City Parkway,Y,Y,768 City Parkway,None,None
6,1007,Jeff,Winger,000-000-0000,1209 South Street,N,N,1209 South Street,None,None
7,1008,Sherlock,Holmes,876-678-3469,98 Clue Drive,N,N,98 Clue Drive,None,None
8,1009,Gandalf,nan,000-000-0000,123 Middle Earth,NaN,NaN,123 Middle Earth,None,None
9,1010,Peter,Parker,123-545-5421,"25th Main Street, New York",N,N,25th Main Street,New York,None
